# 鏡頭校正

In [ ]:
import numpy as np
import cv2
import glob
import pickle

# 定義所使用的棋盤格格式
chessboardSize = (9,6)
# 亞像素計算的迭代終止條件
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# 棋盤座標矩陣序列矩陣創立
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

# 棋盤格單格在真實圖片中的寬度大小
size_of_chessboard_squares_mm = 19
objp = objp * size_of_chessboard_squares_mm

#分別創建用於儲存真實座標與對應像素座標的陣列，陣列中的每個元素代表的是每張圖對應到一組棋盤座標與圖片像素座標
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# 抓取指定路徑所有檔案路徑，輸出為list[str]
images = glob.glob('your/images/location/*.png')
# images = glob.glob('camera/images/*.png')

for result_img in images:

    img = cv2.imread(result_img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 尋找棋盤格角點
    ret, corners = cv2.findChessboardCorners(gray, chessboardSize, None)

    # 若尋找成功就將對應的棋盤座標點與像素點新增近objpoints和imgpoints list當中
    if ret == True:

        objpoints.append(objp)
        # 將找到的圖像像素座標點進行亞像素修正處理，以取得更精確的像素座標位置
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

        cv2.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(1000)


cv2.destroyAllWindows()


# 使用讀取到的圖片座標點與對應的棋盤座標進行鏡頭校正
ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# 儲存鏡頭內參成pkl檔，以便之後調用，而無需重新計算鏡頭內參矩陣與畸變係數
pickle.dump((cameraMatrix, dist), open( "calibration.pkl", "wb" ))

# 桌面座標系建立

### 視窗建立函數

In [ ]:
import tkinter as tk

# 創建獲取長寬表單視窗
def setHeightWin():
    root = tk.Tk()
    root.title("Input Form")

    # 獲取螢幕寬高
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    # 設置視窗寬高(像素)
    window_width = 400  
    window_height = 150 

    # 計算視窗左上角座標
    x = (screen_width - window_width) // 2
    y = (screen_height - window_height) // 2

    # 視窗生成
    root.geometry(f"{window_width}x{window_height}+{x}+{y}")

    # 創建label, entry元件顯示讀取參數
    label_height = tk.Label(root, text="Height:", font=("Helvetica", 14))
    label_height.pack()

    entry_height = tk.Entry(root, font=("Helvetica", 14),justify="center")
    entry_height.pack()

    label_width = tk.Label(root, text="Width:", font=("Helvetica", 14))
    label_width.pack()

    entry_width = tk.Entry(root, font=("Helvetica", 14),justify="center")
    entry_width.pack()
    
    result = []
    # 處理送出的表單
    def submit_form():
        height = entry_height.get()
        width = entry_width.get()
        result.extend([height, width])

        if any(item == "" for item in result):# 當缺少參數的時候就將參數清空，不採取動作
            result.clear()
            return
        # 參數沒問題就銷毀視窗，回傳數值
        root.destroy()
    # 將enter綁定送出表單
    root.bind('<Return>', lambda event=None: submit_form())
    # 創建"送出"按鈕
    submit_button = tk.Button(root, text="送出", command=submit_form,width=10,height=1,font=("Helveticsa",14))
    submit_button.pack()

    # 啟動Tkinter主循環
    root.mainloop()
    return result

### 主程式

In [ ]:
import cv2

#讀取寬高參數
square_height,square_width = setHeightWin()
square_object_points=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])

with open('camera/calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)

image = cv2.imread('pnp_test_img/img101.png')  # 載入圖片
height =480
scale = height / image.shape[0]
image = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
drawing_image = image.copy()
Adjusted_image = None 

isDrawing = True
isAdjusting = False
isFirstDraw = True
square_image_points = []
line_thickness = 2
# 回調函數，處理滑鼠事件
def draw_line(event, x, y, flags, param):
    global line_start, line_end, isDrawing,drawing_image,Adjusted_image,square_image_points,isAdjusting,isFirstDraw,line_thickness
    #偵測事件
    if event == cv2.EVENT_LBUTTONDOWN:  # 當按下左鍵
        if isDrawing:
            if isFirstDraw:
                isFirstDraw = False
            else :
                cv2.line(drawing_image,square_image_points[-1],(x,y),(255,0,0),line_thickness)
            if len(square_image_points) < 4:
                square_image_points.append((x,y))
            if len(square_image_points)==4:
                isDrawing = False
                isAdjusting = True
                frame_update()
        elif isAdjusting:
            distances = np.linalg.norm(np.array(square_image_points) - np.array((x,y)), axis=1)
            nearest_index = np.argmin(distances)
            square_image_points[nearest_index]=(x,y)
            frame_update()
    elif event == cv2.EVENT_MOUSEMOVE:  # 當滑鼠移動
        if isDrawing and not isFirstDraw:
            temp_image = drawing_image.copy()  # 複製圖像而非變更原圖
            cv2.line(temp_image, square_image_points[-1], (x, y), (0, 255, 0), line_thickness)
            cv2.imshow('Set Table axis', temp_image)
        if isAdjusting:
            cv2.imshow('Set Table axis', Adjusted_image)

    elif event == cv2.EVENT_MOUSEWHEEL: # 當滑鼠滾輪滾動
        delta = flags
        if delta > 0:
            if line_thickness==3:
                pass
            else:
                line_thickness+=1
        elif delta < 0:
            if line_thickness == 1:
                pass
            else:
                line_thickness-=1
        frame_update()

# 創建視窗
cv2.namedWindow('Set Table axis')
# 綁定監聽滑鼠事件，綁定處理事件回調函數
cv2.setMouseCallback('Set Table axis', draw_line)

def frame_update(): # 進行畫面更新
    global Adjusted_image, square_image_points,line_thickness,rvecs,tvecs
    Adjusted_image=image.copy()
    cv2.polylines(Adjusted_image, [np.array(square_image_points)], isClosed=True, color=(0, 255, 0), thickness=line_thickness)
    for coordinate,point in zip(square_object_points,square_image_points):
        cv2.putText(Adjusted_image, f"{list(coordinate)}", point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0, 255), 2)
    square_img_points = np.float32(square_image_points)
    # print(square_img_points)
    ret,rvecs,tvecs = cv2.solvePnP(square_object_points,square_img_points,mtx,dist)
    # print(rvecs)
    axis = np.float32([[100,0,0], [0,100,0], [0,0,100]]).reshape(-1,3)
    axis_img_points, _ = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
    draw_axis(Adjusted_image,square_img_points.astype(int),axis_img_points)

# 顯示圖像
while True:
    if not isAdjusting:
        cv2.imshow('Set Table axis', drawing_image)
    else:
        cv2.imshow('Set Table axis', Adjusted_image)
    key = cv2.waitKey(1)
    if key == 27:  # 當按下ESC退出循環
        break
    if len(square_image_points)!=4: #確認是否四個像素座標點抓到了
        continue
    if key == ord("r"): # Z軸對調
        square_image_points[1],square_image_points[3]=square_image_points[3],square_image_points[1]
        # square_coordinates[1],square_coordinates[3]=square_coordinates[3],square_coordinates[1]
        frame_update()
    if key == ord("e"): # 變更原點
        square_image_points=square_image_points[-1:]+ square_image_points[:-1] #前包後不包
        frame_update()
    if key == ord("w"): # 重設長寬
        square_height,square_width = setHeightWin()
        square_object_points=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])
        frame_update()
    if key ==ord("q"): # 長寬互換
        square_height,square_width=square_width,square_height
        square_object_points=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])
        frame_update()
    if key == ord("s"): # 儲存真實與圖像座標和PNP計算結果
        with open('table_points.pkl', 'wb') as file:
                pickle.dump((square_object_points,np.float32(square_image_points)), file)
        with open('table_solvePnP.pkl', 'wb') as file:
                pickle.dump((rvecs,tvecs), file)
        print("saved")
                    
cv2.destroyAllWindows()